In [1]:
def dy_dx(x):
    return 2 * x

In [2]:
import torch
dy_dx(3)

6

In [3]:
x = torch.tensor(3.0, requires_grad=True)
y = x ** 2

In [4]:
print(x)
print(y)

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)


In [5]:
y.backward()

In [6]:
x.grad

tensor(6.)